In [13]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Lambda, Add, Concatenate, Layer
from tensorflow.keras.optimizers import Adam

import gymnasium as gym
import ale_py
gym.register_envs(ale_py)

from collections import deque
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [18]:
# Constants
ENV_NAME = "ALE/Breakout-v5"
FRAMES_IN_STATE = 3  # Number of consecutive frames to stack for state representation
FRAME_SIZE = (84, 84)  # Size of each frame after preprocessing
BATCH_SIZE = 32
REPLAY_BUFFER_SIZE = 100000
GAMMA = 0.99  # Discount factor
EPSILON_START = 1.0
EPSILON_END = 0.1
EPSILON_DECAY_STEPS = 1000000
LEARNING_RATE = 0.00025
TARGET_UPDATE_FREQ = 10000  # Steps between target network updates
TRAIN_FREQ = 4  # Steps between training
SAVE_MODEL_FREQ = 50000  # Steps between saving model checkpoints
TOTAL_STEPS = 5000000  # Total steps to train for
REPLAY_START_SIZE = 50000  # Steps to populate replay buffer before training starts
SAVE_DIR = "breakout_dqn_model"
PLOT_FREQUENCY = 10000  # Steps between plotting progress

class PreprocessAtari:
    """Class to preprocess Atari frames."""
    def __init__(self, frame_size=FRAME_SIZE):
        self.frame_size = frame_size
        
    def process(self, frame):
        """Convert RGB to grayscale, crop, and resize."""
        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        
        # Crop the frame (remove the score at the top)
        cropped = gray[34:194, :]
        
        # Resize the frame
        resized = cv2.resize(cropped, self.frame_size, interpolation=cv2.INTER_AREA)
        
        # Normalize pixel values
        normalized = resized / 255.0
        
        return normalized

class FrameStack:
    """Class to manage frame stacking for state representation."""
    def __init__(self, num_frames=FRAMES_IN_STATE, frame_size=FRAME_SIZE):
        self.num_frames = num_frames
        self.frames = deque(maxlen=num_frames)
        self.frame_size = frame_size
        
    def reset(self, initial_frame):
        """Reset the frame stack with initial frame."""
        self.frames.clear()
        processed_frame = PreprocessAtari(self.frame_size).process(initial_frame)
        for _ in range(self.num_frames):
            self.frames.append(processed_frame)
        return self.get_state()
    
    def add_frame(self, frame):
        """Add a new frame to the stack."""
        processed_frame = PreprocessAtari(self.frame_size).process(frame)
        self.frames.append(processed_frame)
        return self.get_state()
    
    def get_state(self):
        """Get the current state as stacked frames."""
        return np.stack(self.frames, axis=-1)

class ReplayBuffer:
    """Experience replay buffer to store and sample transitions."""
    def __init__(self, capacity=REPLAY_BUFFER_SIZE):
        self.buffer = deque(maxlen=capacity)
        
    def add(self, state, action, reward, next_state, done):
        """Add a transition to the buffer."""
        self.buffer.append((state, action, reward, next_state, done))
        
    def sample(self, batch_size):
        """Sample a batch of transitions."""
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return (
            np.array(states, dtype=np.float32),
            np.array(actions, dtype=np.int32),
            np.array(rewards, dtype=np.float32),
            np.array(next_states, dtype=np.float32),
            np.array(dones, dtype=np.bool_)
        )
        
    def __len__(self):
        return len(self.buffer)

class DuelingLayer(Layer):
    """Custom Keras layer implementing the dueling network architecture."""
    def __init__(self, num_actions, **kwargs):
        super(DuelingLayer, self).__init__(**kwargs)
        self.num_actions = num_actions
        self.dense_value = Dense(256, activation='relu')
        self.dense_advantage = Dense(256, activation='relu')
        self.value = Dense(1)
        self.advantage = Dense(num_actions)
    
    def call(self, inputs):
        # Value stream
        value_stream = self.dense_value(inputs)
        value = self.value(value_stream)
        
        # Advantage stream
        advantage_stream = self.dense_advantage(inputs)
        advantage = self.advantage(advantage_stream)
        
        # Combine value and advantage streams
        advantage_mean = tf.reduce_mean(advantage, axis=1, keepdims=True)
        q_values = value + (advantage - advantage_mean)
        
        return q_values

def build_dueling_dqn(input_shape, num_actions):
    inputs = Input(shape=input_shape)
    
    # Convolutional layers
    x = Conv2D(32, (8, 8), strides=4, activation='relu')(inputs)
    x = Conv2D(64, (4, 4), strides=2, activation='relu')(x)
    x = Conv2D(64, (3, 3), strides=1, activation='relu')(x)
    x = Flatten()(x)
    
    # Common feature layer
    features = Dense(512, activation='relu')(x)
    
    # Dueling layer that implements the value and advantage streams
    outputs = DuelingLayer(num_actions)(features)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

class DoubleDuelingDQNAgent:
    """Agent implementing Double Dueling DQN."""
    def __init__(self, state_shape, num_actions):
        self.state_shape = state_shape
        self.num_actions = num_actions
        
        # Create main and target networks
        self.main_network = build_dueling_dqn(state_shape, num_actions)
        self.target_network = build_dueling_dqn(state_shape, num_actions)
        self.target_network.set_weights(self.main_network.get_weights())
        
        # Compile the model
        self.main_network.compile(optimizer=Adam(learning_rate=LEARNING_RATE))
        
        # Create replay buffer
        self.replay_buffer = ReplayBuffer()
        
        # Exploration parameters
        self.epsilon = EPSILON_START
        self.epsilon_decay = (EPSILON_START - EPSILON_END) / EPSILON_DECAY_STEPS
        self.epsilon_min = EPSILON_END
        
        # Tracking variables
        self.step_count = 0
        self.training_step = 0
        
        # Create directories for saving models
        if not os.path.exists(SAVE_DIR):
            os.makedirs(SAVE_DIR)
            
        # Metrics
        self.rewards_history = []
        self.episode_lengths = []
        self.episode_count = 0
        self.running_reward = 0
        self.best_reward = float('-inf')
        self.losses = []
        
    def get_action(self, state, training=True):
        """Get an action using epsilon-greedy policy."""
        if training and random.random() < self.epsilon:
            return random.randrange(self.num_actions)
        
        q_values = self.main_network.predict(np.expand_dims(state, axis=0), verbose=0)[0]
        return np.argmax(q_values)
    
    def update_epsilon(self):
        """Decay epsilon value."""
        self.epsilon = max(self.epsilon_min, self.epsilon - self.epsilon_decay)
        
    def train(self, batch):
        """Train the model on a batch of experiences."""
        states, actions, rewards, next_states, dones = batch
        
        # Get the actions from the main network for double DQN
        next_actions = np.argmax(self.main_network.predict(next_states, verbose=0), axis=1)
        
        # Get Q-values from the target network
        next_q_values = self.target_network.predict(next_states, verbose=0)
        
        # Select Q-values for the actions chosen by the main network
        target_q_values = np.zeros_like(rewards)
        for i in range(len(rewards)):
            if dones[i]:
                target_q_values[i] = rewards[i]
            else:
                target_q_values[i] = rewards[i] + GAMMA * next_q_values[i, next_actions[i]]
        
        # Create a mask for the actions that were taken
        masks = tf.one_hot(actions, self.num_actions)
        
        with tf.GradientTape() as tape:
            q_values = self.main_network(states)
            
            # Apply the masks to get the Q-values for the actions taken
            q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
            
            # Calculate loss
            loss = tf.reduce_mean(tf.square(target_q_values - q_action))
            
        # Backpropagation
        grads = tape.gradient(loss, self.main_network.trainable_variables)
        self.main_network.optimizer.apply_gradients(zip(grads, self.main_network.trainable_variables))
        
        return loss.numpy()
    
    def update_target_network(self):
        """Update weights of the target network."""
        self.target_network.set_weights(self.main_network.get_weights())
        
    def save_model(self, name="model"):
        """Save the model weights."""
        self.main_network.save_weights(f"{SAVE_DIR}/{name}.weights.h5")
        
    def load_model(self, name="model"):
        """Load the model weights."""
        self.main_network.load_weights(f"{SAVE_DIR}/{name}.weights.h5")
        self.target_network.set_weights(self.main_network.get_weights())
        
    def plot_metrics(self):
        """Plot training metrics."""
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 3, 1)
        plt.plot(self.rewards_history)
        plt.title('Episode Rewards')
        plt.xlabel('Episode')
        plt.ylabel('Total Reward')
        
        plt.subplot(1, 3, 2)
        plt.plot(self.episode_lengths)
        plt.title('Episode Lengths')
        plt.xlabel('Episode')
        plt.ylabel('Steps')
        
        if self.losses:
            plt.subplot(1, 3, 3)
            plt.plot(self.losses)
            plt.title('Training Loss')
            plt.xlabel('Training Step')
            plt.ylabel('Loss')
        
        plt.tight_layout()
        plt.savefig(f"{SAVE_DIR}/metrics.png")
        plt.close()

def train_agent():
    """Train the agent on the Atari Breakout environment."""
    env = gym.make(ENV_NAME)
    
    state_shape = (*FRAME_SIZE, FRAMES_IN_STATE)
    num_actions = env.action_space.n
    
    agent = DoubleDuelingDQNAgent(state_shape, num_actions)
    frame_stack = FrameStack()
    preprocessor = PreprocessAtari()
    
    # Variables for tracking
    episode_reward = 0
    episode_steps = 0
    state = frame_stack.reset(env.reset()[0])
    
    # Progress bar
    pbar = tqdm(total=TOTAL_STEPS, desc="Training")
    
    # Main training loop
    while agent.step_count < TOTAL_STEPS:
        # Get action
        action = agent.get_action(state)
        
        # Take step in environment
        next_frame, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        
        # Process next state
        next_state = frame_stack.add_frame(next_frame)
        
        # Store in replay buffer
        agent.replay_buffer.add(state, action, reward, next_state, done)
        
        # Update state
        state = next_state
        
        # Track episode stats
        episode_reward += reward
        episode_steps += 1
        agent.step_count += 1
        pbar.update(1)
        
        # Update epsilon
        agent.update_epsilon()
        
        # Train if replay buffer is large enough and it's time to train
        if len(agent.replay_buffer) > BATCH_SIZE and agent.step_count > REPLAY_START_SIZE and agent.step_count % TRAIN_FREQ == 0:
            batch = agent.replay_buffer.sample(BATCH_SIZE)
            loss = agent.train(batch)
            agent.losses.append(loss)
            agent.training_step += 1
            
        # Update target network periodically
        if agent.step_count % TARGET_UPDATE_FREQ == 0 and agent.step_count > REPLAY_START_SIZE:
            agent.update_target_network()
            print(f"\nUpdated target network at step {agent.step_count}")
        
        # Save model periodically
        if agent.step_count % SAVE_MODEL_FREQ == 0 and agent.step_count > REPLAY_START_SIZE:
            agent.save_model(f"model_step_{agent.step_count}")
            print(f"\nSaved model at step {agent.step_count}")
        
        # Plot metrics periodically
        if agent.step_count % PLOT_FREQUENCY == 0 and agent.step_count > REPLAY_START_SIZE:
            agent.plot_metrics()
            
        # Reset environment if episode is done
        if done:
            # Track episode metrics
            agent.rewards_history.append(episode_reward)
            agent.episode_lengths.append(episode_steps)
            agent.episode_count += 1
            
            # Update running reward
            if agent.running_reward == 0:
                agent.running_reward = episode_reward
            else:
                agent.running_reward = 0.05 * episode_reward + 0.95 * agent.running_reward
                
            # Save best model
            if episode_reward > agent.best_reward:
                agent.best_reward = episode_reward
                agent.save_model("best_model")
                
            # Print episode info
            print(f"\nEpisode {agent.episode_count} - Reward: {episode_reward}, Steps: {episode_steps}, Epsilon: {agent.epsilon:.4f}, Running Reward: {agent.running_reward:.2f}")
            
            # Reset episode variables
            episode_reward = 0
            episode_steps = 0
            state = frame_stack.reset(env.reset()[0])
    
    # Close environment and progress bar
    env.close()
    pbar.close()
    
    # Final save and plot
    agent.save_model("final_model")
    agent.plot_metrics()
    
    return agent

def evaluate_agent(model_path="final_model", episodes=10, render=False):
    """Evaluate the trained agent."""
    env = gym.make(ENV_NAME, render_mode="human" if render else None)
    
    state_shape = (*FRAME_SIZE, FRAMES_IN_STATE)
    num_actions = env.action_space.n
    
    agent = DoubleDuelingDQNAgent(state_shape, num_actions)
    agent.load_model(model_path)
    agent.epsilon = 0.0  # No exploration during evaluation
    
    frame_stack = FrameStack()
    
    total_rewards = []
    
    for episode in range(episodes):
        state = frame_stack.reset(env.reset()[0])
        episode_reward = 0
        done = False
        
        while not done:
            action = agent.get_action(state, training=False)
            next_frame, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            state = frame_stack.add_frame(next_frame)
            episode_reward += reward
            
        total_rewards.append(episode_reward)
        print(f"Episode {episode+1} - Reward: {episode_reward}")
    
    env.close()
    
    print(f"Average Reward over {episodes} episodes: {np.mean(total_rewards):.2f}")
    return total_rewards

In [ ]:
# Train the agent
print("Starting training...")
agent = train_agent()

Starting training...


Training:   0%|          | 268/5000000 [00:00<3:29:41, 397.38it/s]


Episode 1 - Reward: 2.0, Steps: 200, Epsilon: 0.9998, Running Reward: 2.00


Training:   0%|          | 460/5000000 [00:00<2:55:18, 475.31it/s]


Episode 2 - Reward: 3.0, Steps: 231, Epsilon: 0.9996, Running Reward: 2.05


...

Training:   1%|          | 56553/5000000 [10:44<111:42:00, 12.29it/s]


Episode 299 - Reward: 2.0, Steps: 239, Epsilon: 0.9491, Running Reward: 1.52


Training:   1%|          | 56649/5000000 [10:53<114:38:16, 11.98it/s]

KeyboardInterrupt: 

Training:   1%|          | 56652/5000000 [11:04<114:38:16, 11.98it/s]

In [ ]:
# Evaluate the trained agent
print("\nEvaluating the trained agent...")
evaluate_agent("best_model", episodes=5, render=True)